In [82]:
# Causual inference of price elasticity of demand is the process of identifying cause-effect relationships between variables.
# How to interpret value of price elasticity of demand ?
    # |PED| <1 - inelastic (demand LESS sensitive to price changes)
    # |PED| >1 - elastic (demand MORE sensitive to price changes)
    # +ve - demand increase with price increase
    # -ve - demand decrease with price increase (typical case)
# Usually, the sign of elasticity value is ignored for easier interpretation. We assume inverse relationship of demand and price
# Treatment variable: price
# Outcome variable: demand

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

##### Derive log transformed PRICE for PED estimation

In [89]:
# Read data
data = pd.read_excel("Dataset/FINAL SUBSET_preprocessed_data.xlsx")  
df1 = data.copy()

# Derive log transformed PRICE
df1['PRICE_LOG'] = np.log(df1['PRICE'])
df1[['PRICE', 'PRICE_LOG']]

,PRICE,PRICE_LOG
0,1.39,0.329304
1,1.39,0.329304
2,1.38,0.322083
3,2.50,0.916291
4,2.59,0.951658
...,...,...
414932,4.97,1.603420
414933,4.97,1.603420
414934,3.31,1.196948
414935,3.31,1.196948


In [91]:
temp = df1[(df1['STORE_ID'] == 389) & (df1['UPC'] == 1111038078)]
temp[['WEEK_END_DATE', 'YEAR', 'WEEK_NUM', 'STORE_ID', 'UPC', 'UNIT_SALES', 'PRICE']]

,WEEK_END_DATE,YEAR,WEEK_NUM,STORE_ID,UPC,UNIT_SALES,PRICE
64,2009-01-14,2009,3,389,1111038078,6,2.59
2579,2009-01-21,2009,4,389,1111038078,28,0.98
5083,2009-01-28,2009,5,389,1111038078,24,1.00
7590,2009-02-04,2009,6,389,1111038078,13,1.00
10065,2009-02-11,2009,7,389,1111038078,16,1.00
...,...,...,...,...,...,...,...
402408,2011-12-07,2011,49,389,1111038078,11,1.04
404912,2011-12-14,2011,50,389,1111038078,6,1.69
407434,2011-12-21,2011,51,389,1111038078,4,1.69
409979,2011-12-28,2011,52,389,1111038078,5,1.69


#### Double Machine Learning Model Building

In [3]:
# Define confounding variables
confounders =['YEAR','WEEK_NUM', 'FEATURE', 'DISPLAY', 'SEG_VALUE_NAME_ORDINAL', 'SALES_AREA_SIZE_NUM'] 

treatment_r2_scores = []
treatment_mae_scores = []
outcome_r2_scores = []
outcome_mae_scores = []

def estimate_price_elasticity(store_id, upc):
    global treatment_r2_scores, treatment_mae_scores, outcome_r2_scores, outcome_mae_scores

    # Extract subset data for each store-upc combination
    subset_df = df1[(df1['STORE_ID'] == store_id) & (df1['UPC'] == upc)]

    # Define treatment (PRICE) and outcome variable (UNIT SALES)
    X_train_treatment = subset_df[confounders] 
    X_train_outcome = subset_df[['PRICE_LOG'] + confounders] 

    Y_treatment = subset_df['PRICE_LOG']
    Y_outcome = subset_df['UNIT_SALES_LOG']

    # First stage estimation: Regress treatment (PRICE_LOG) and outcome (UNIT_SALES_LOG) on cofounders
    treatment_model = XGBRegressor(random_state=42)   
    outcome_model = XGBRegressor(random_state=42)
    
    treatment_model.fit(X_train_treatment, Y_treatment)
    outcome_model.fit(X_train_outcome, Y_outcome)
 
    Y_treatment_pred = treatment_model.predict(X_train_treatment)
    Y_outcome_pred = outcome_model.predict(X_train_outcome)

    # Derive residuals to approximate percentage changes of PRICE and UNIT_SALES. 
    log_price_residuals = Y_treatment - Y_treatment_pred
    log_demand_residuals = Y_outcome - Y_outcome_pred

    # print("R² (Treatment model): ", r2_score(np.exp(Y_treatment), np.exp(Y_treatment_pred)))
    # print("MAE (Treatment model): ", mean_absolute_error(np.exp(Y_treatment), np.exp(Y_treatment_pred)))
    # print("R² (Ouctome model): ", r2_score(np.exp(Y_outcome), np.exp(Y_outcome_pred)))
    # print("MAE (Ouctome model): ", mean_absolute_error(np.exp(Y_outcome), np.exp(Y_outcome_pred)))
    
    treatment_r2 = r2_score(np.exp(Y_treatment), np.exp(Y_treatment_pred))
    treatment_mae = mean_absolute_error(np.exp(Y_treatment), np.exp(Y_treatment_pred))
    treatment_r2_scores.append(treatment_r2)
    treatment_mae_scores.append(treatment_mae)

    outcome_r2 = r2_score(np.exp(Y_outcome), np.exp(Y_outcome_pred))
    outcome_mae = mean_absolute_error(np.exp(Y_outcome), np.exp(Y_outcome_pred))
    outcome_r2_scores.append(outcome_r2)
    outcome_mae_scores.append(outcome_mae)

    # Second stage estimation [log-log regression model]: Regress treatment (X) and outcome residulas (Y)  
    log_linear_model = LinearRegression()
    log_linear_model.fit(log_price_residuals.values.reshape(-1, 1), log_demand_residuals) # reshape input X to 2D array
    price_elasticity = log_linear_model.coef_[0]

    return price_elasticity 

#### Price Elasticify of Demand Estimation (Loop each store-product pair)

In [28]:
# Extract unique store-upc combinations 
unique_store_upc = df1[['STORE_ID', 'UPC']].drop_duplicates()
unique_store_upc['PRICE_ELASTICITY'] = np.nan
print(len(unique_store_upc), "unique store-upc combinations\n")

for index, row in unique_store_upc.iterrows():
    store_id = int(row['STORE_ID'])
    upc = int(row['UPC'])
    price_elasticity = estimate_price_elasticity(store_id, upc)
    print(price_elasticity)
    unique_store_upc.at[index, 'PRICE_ELASTICITY'] = price_elasticity

# For interpretability, use the absolute value. Ignore the sign by assuming all products follow Law of Demand (price increase, demand decrease)
unique_store_upc['ABS_PRICE_ELASTICITY'] = unique_store_upc['PRICE_ELASTICITY'].abs()   
unique_store_upc = unique_store_upc.sort_values(by=['UPC', 'STORE_ID'])

#unique_store_upc.to_excel('Dataset/Price Elasticity.xlsx', index=False)

2845 unique store-upc combinations

0.06387128623989084
-0.10083786571282213
0.47913501213850107
0.10915926307388966
0.008482251935989829
0.0873857489541831
-0.04667944091544525
0.23136302655632618
0.08508002914825981
-0.09605969851032874
0.02450798898264059
0.02381131944301787
-0.16661760224096048
0.12498540636596601
-0.985131525543551
0.3616472743895571
0.4788640431795879
0.04367717526788818
-0.13083387801740748
0.20190919618367356
0.25817649079310284
0.2537166146736139
0.03776709559142627
-0.4803926347057593
-0.7031433450925346
-0.018307940226353018
-0.22502313378254443
0.19123374465351148
-0.06930128425757381
0.05881056795718357
-0.13439461856333185
0.04914581073368924
0.13224496686816234
0.22019578874569823
0.10537078920534285
0.041416578494487356
0.0470093834550636
-0.09593612422945576
0.15048005249885238
0.07283358341473965
-0.14320195710484757
0.11373808606297993
-0.2715914307541507
-0.2054719129442682
0.051427036403574856
-0.09177381717753229
-0.16519456073776292
-0.0595205929

In [4]:
def get_average_metrics():
    avg_treatment_r2 = np.mean(treatment_r2_scores)
    avg_treatment_mae = np.mean(treatment_mae_scores)
    avg_outcome_r2 = np.mean(outcome_r2_scores)
    avg_outcome_mae = np.mean(outcome_mae_scores)
    return avg_treatment_r2, avg_treatment_mae, avg_outcome_r2, avg_outcome_mae  

In [29]:
avg_treatment_r2, avg_treatment_mae, avg_outcome_r2, avg_outcome_mae = get_average_metrics()
print("Treatment Model")
print("Average R²: ",avg_treatment_r2)
print("Average MAE: ", avg_treatment_mae)

print("\nOutcome Model")
print("Average R²: ", avg_outcome_r2)
print("Average MAE: ", avg_outcome_mae)

Treatment Model
Average R²:  0.9980169936191255
Average MAE:  0.007128422690295444

Outcome Model
Average R²:  0.99962591754747
Average MAE:  0.10788821180782143


#### Results

In [2]:
elasticity = pd.read_excel('Dataset/Price Elasticity.xlsx')
product = pd.read_excel('Dataset/Product Details.xlsx')
store = pd.read_excel('Dataset/Store Details.xlsx')
elasticity

,STORE_ID,UPC,PRICE_ELASTICITY,ABS_PRICE_ELASTICITY
0,367,1111009477,0.063871,0.063871
1,387,1111009477,0.058811,0.058811
2,389,1111009477,0.009605,0.009605
3,613,1111009477,0.071135,0.071135
4,623,1111009477,-0.028330,0.028330
...,...,...,...,...
2840,26981,88491212971,0.082167,0.082167
2841,26983,88491212971,0.050827,0.050827
2842,27175,88491212971,-0.047577,0.047577
2843,28909,88491212971,0.164867,0.164867


In [3]:
print("Total Store-Product Pairs: ", len(elasticity), '\n')  # 2845
elasticity[elasticity['PRICE_ELASTICITY'] < 0]  # 1385 (follow law of demand)
elasticity[elasticity['PRICE_ELASTICITY'] > 0]  # 1459 (contrast law of demand)

perfectly_inelastic = elasticity[elasticity['ABS_PRICE_ELASTICITY'] == 0] #(perfectly inelastic: price changes have no impact on demand)
print("Perfectly Inelastic Demand (=0): ", len(perfectly_inelastic)) 

inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0) & (elasticity['ABS_PRICE_ELASTICITY'] < 1)]
print("Inelastic Demand (<1): ", len(inelastic_store_product))   

highly_inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0) & (elasticity['ABS_PRICE_ELASTICITY'] <= 0.1)]
print("Highly Inelastic Demand (>0 and <=0.1): ", len(highly_inelastic_store_product))   

moderately_inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0.1) & (elasticity['ABS_PRICE_ELASTICITY'] <= 0.5)]
print("Moderately Inelastic Demand (>0.1 and <=0.5): ", len(moderately_inelastic_store_product))  

slightly_inelastic_store_product = elasticity[(elasticity['ABS_PRICE_ELASTICITY'] > 0.5) & (elasticity['ABS_PRICE_ELASTICITY'] < 1)]
print("Slightly Inelastic Demand (>0.5 and <1): ", len(slightly_inelastic_store_product))  

elastic_store_product = elasticity[elasticity['ABS_PRICE_ELASTICITY'] > 1]   
print("Elastic Demand (>1): ", len(elastic_store_product))   # 44 store-product

Total Store-Product Pairs:  2845 

Perfectly Inelastic Demand (=0):  1
Inelastic Demand (<1):  2800
Highly Inelastic Demand (>0 and <=0.1):  1341
Moderately Inelastic Demand (>0.1 and <=0.5):  1272
Slightly Inelastic Demand (>0.5 and <1):  187
Elastic Demand (>1):  44


In [3]:
# perfectly_inelastic
data[(data['STORE_ID'] == 2523) & (data['UPC'] == 7797502248)]

,WEEK_END_DATE,YEAR,WEEK_NUM,STORE_ID,UPC,UNIT_SALES,UNIT_SALES_LOG,PRICE,PRICE_SQRT,BASE_PRICE,...,DISCOUNT_RATE,FEATURE,DISPLAY,PROMOTION_SUPPORT,CATEGORY,SUB_CATEGORY,STORE_NAME,SEG_VALUE_NAME,SEG_VALUE_NAME_ORDINAL,SALES_AREA_SIZE_NUM
277423,2011-01-12,2011,2,2523,7797502248,1,0.0,2.49,1.577973,2.49,...,0.0,0,0,NONE,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
282946,2011-01-26,2011,4,2523,7797502248,1,0.0,2.49,1.577973,2.49,...,0.0,0,0,NONE,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
296653,2011-03-02,2011,9,2523,7797502248,1,0.0,2.49,1.577973,2.49,...,0.0,0,0,NONE,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
299382,2011-03-09,2011,10,2523,7797502248,1,0.0,2.49,1.577973,2.49,...,0.0,0,0,NONE,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
304894,2011-03-23,2011,12,2523,7797502248,1,0.0,2.49,1.577973,2.49,...,0.0,0,0,NONE,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373
313135,2011-04-13,2011,15,2523,7797502248,1,0.0,2.49,1.577973,2.49,...,0.0,0,0,NONE,BAG SNACKS,PRETZELS,LANDMARK PLACE S/C,VALUE,0,34373


In [4]:
def get_store_product(store_id, upc):
    store_row = store[store['STORE_ID'] == store_id]
    store_info = {
        'STORE_ID': int(store_id),
        'STORE_NAME': store_row['STORE_NAME'].iloc[0],
        'STORE_SEGMENT_VALUE': store_row['SEG_VALUE_NAME'].iloc[0]
    }
    upc_row = product[product['UPC'] == upc]
    product_info = {
        'UPC': int(upc),
        'PRODUCT_CATEGORY': upc_row['CATEGORY'].iloc[0],
        'PRODUCT_SUB_CATEGORY': upc_row['SUB_CATEGORY'].iloc[0],
        'PRODUCT_MANUFACTURER': upc_row['MANUFACTURER'].iloc[0]
    }
    store_product_info = {**store_info, **product_info}
    return store_product_info

In [5]:
highly_inelastic_store_product_ls = []
for index, row in highly_inelastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    highly_inelastic_store_product_ls.append(store_product_info)
highly_inelastic_store_product_df = pd.DataFrame(highly_inelastic_store_product_ls)


moderately_inelastic_store_product_ls = []
for index, row in moderately_inelastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    moderately_inelastic_store_product_ls.append(store_product_info)
moderately_inelastic_store_product_df = pd.DataFrame(moderately_inelastic_store_product_ls)


slightly_inelastic_store_product_ls = []
for index, row in slightly_inelastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    slightly_inelastic_store_product_ls.append(store_product_info)
slightly_inelastic_store_product_df = pd.DataFrame(slightly_inelastic_store_product_ls)


elastic_store_product_ls = []
for index, row in elastic_store_product.reset_index().iterrows():
    store_id = row['STORE_ID']
    upc = row['UPC']
    store_product_info = get_store_product(store_id, upc)
    elastic_store_product_ls.append(store_product_info)
elastic_store_product_df = pd.DataFrame(elastic_store_product_ls)

In [6]:
print(highly_inelastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(highly_inelastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

COLD CEREAL              620
BAG SNACKS               285
FROZEN PIZZA             264
ORAL HYGIENE PRODUCTS    172
Name: PRODUCT_CATEGORY, dtype: int64

ALL FAMILY CEREAL              331
PRETZELS                       285
PIZZA/PREMIUM                  264
KIDS CEREAL                    172
MOUTHWASHES (ANTISEPTIC)       152
ADULT CEREAL                   117
MOUTHWASH/RINSES AND SPRAYS     20
Name: PRODUCT_SUB_CATEGORY, dtype: int64


In [23]:
highly_inelastic_store_product_df[(highly_inelastic_store_product_df['PRODUCT_CATEGORY'] == 'FROZEN PIZZA') 
                                  & (highly_inelastic_store_product_df['STORE_ID'] == 387)]

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
407,387,TOWN & COUNTRY,UPSCALE,1111087395,FROZEN PIZZA,PIZZA/PREMIUM,PRIVATE LABEL
469,387,TOWN & COUNTRY,UPSCALE,1111087396,FROZEN PIZZA,PIZZA/PREMIUM,PRIVATE LABEL
1021,387,TOWN & COUNTRY,UPSCALE,7192100337,FROZEN PIZZA,PIZZA/PREMIUM,TOMBSTONE
1041,387,TOWN & COUNTRY,UPSCALE,7192100339,FROZEN PIZZA,PIZZA/PREMIUM,TOMBSTONE


In [7]:
print(moderately_inelastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(moderately_inelastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

COLD CEREAL              488
ORAL HYGIENE PRODUCTS    296
BAG SNACKS               250
FROZEN PIZZA             238
Name: PRODUCT_CATEGORY, dtype: int64

PRETZELS                       250
MOUTHWASHES (ANTISEPTIC)       250
PIZZA/PREMIUM                  238
KIDS CEREAL                    200
ALL FAMILY CEREAL              187
ADULT CEREAL                   101
MOUTHWASH/RINSES AND SPRAYS     46
Name: PRODUCT_SUB_CATEGORY, dtype: int64


In [18]:
moderately_inelastic_store_product_df[
    (moderately_inelastic_store_product_df['PRODUCT_CATEGORY'] == 'COLD CEREAL') 
    & (moderately_inelastic_store_product_df['STORE_ID'] == 387)]

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
141,387,TOWN & COUNTRY,UPSCALE,1111085319,COLD CEREAL,ALL FAMILY CEREAL,PRIVATE LABEL
160,387,TOWN & COUNTRY,UPSCALE,1111085345,COLD CEREAL,ADULT CEREAL,PRIVATE LABEL
257,387,TOWN & COUNTRY,UPSCALE,1600027528,COLD CEREAL,ALL FAMILY CEREAL,GENERAL MI
302,387,TOWN & COUNTRY,UPSCALE,1600027564,COLD CEREAL,ALL FAMILY CEREAL,GENERAL MI
466,387,TOWN & COUNTRY,UPSCALE,3000006610,COLD CEREAL,KIDS CEREAL,QUAKER
650,387,TOWN & COUNTRY,UPSCALE,3800031838,COLD CEREAL,KIDS CEREAL,KELLOGG
666,387,TOWN & COUNTRY,UPSCALE,3800039118,COLD CEREAL,KIDS CEREAL,KELLOGG
1196,387,TOWN & COUNTRY,UPSCALE,88491201427,COLD CEREAL,ADULT CEREAL,POST FOODS


In [8]:
print(slightly_inelastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(slightly_inelastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

ORAL HYGIENE PRODUCTS    62
BAG SNACKS               51
COLD CEREAL              39
FROZEN PIZZA             35
Name: PRODUCT_CATEGORY, dtype: int64

MOUTHWASHES (ANTISEPTIC)       53
PRETZELS                       51
PIZZA/PREMIUM                  35
ALL FAMILY CEREAL              17
ADULT CEREAL                   11
KIDS CEREAL                    11
MOUTHWASH/RINSES AND SPRAYS     9
Name: PRODUCT_SUB_CATEGORY, dtype: int64


In [52]:
slightly_inelastic_store_product_df[
slightly_inelastic_store_product_df['PRODUCT_CATEGORY'] == 'ORAL HYGIENE PRODUCTS']

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
0,6187,ST. MARYS,MAINSTREAM,1111038078,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
1,13609,VANDALIA,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
2,15547,WOODLAWN,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
3,23067,WHIPP & BIGGER,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
4,26981,SPRINGFIELD,MAINSTREAM,1111038080,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),PRIVATE LABEL
...,...,...,...,...,...,...,...
169,23061,LIBERTY TWP.,MAINSTREAM,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER
170,25001,PINEWOOD,VALUE,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER
171,25233,ANTOINE TOWN CENTER,VALUE,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER
172,25261,CROWLEY,MAINSTREAM,31254742835,ORAL HYGIENE PRODUCTS,MOUTHWASHES (ANTISEPTIC),WARNER


In [9]:
print(elastic_store_product_df['PRODUCT_CATEGORY'].value_counts())
print()
print(elastic_store_product_df['PRODUCT_SUB_CATEGORY'].value_counts())

BAG SNACKS               29
COLD CEREAL               8
ORAL HYGIENE PRODUCTS     5
FROZEN PIZZA              2
Name: PRODUCT_CATEGORY, dtype: int64

PRETZELS                       29
ALL FAMILY CEREAL               4
MOUTHWASHES (ANTISEPTIC)        3
MOUTHWASH/RINSES AND SPRAYS     2
KIDS CEREAL                     2
PIZZA/PREMIUM                   2
ADULT CEREAL                    2
Name: PRODUCT_SUB_CATEGORY, dtype: int64


In [64]:
elastic_store_product_df[(elastic_store_product_df['PRODUCT_CATEGORY'] == 'BAG SNACKS') ]

,STORE_ID,STORE_NAME,STORE_SEGMENT_VALUE,UPC,PRODUCT_CATEGORY,PRODUCT_SUB_CATEGORY,PRODUCT_MANUFACTURER
4,4521,PARKWAY SQUARE S/C,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
5,6187,ST. MARYS,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
6,6431,AT WARD ROAD,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
7,11967,NORTHBOROUGH SQ,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
8,12011,SHERMAN,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
9,13859,MCKINNEY,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
10,15755,KROGER JUNCTION S/C,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
11,17599,KEARNEY,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
12,17615,SUGARLAND,MAINSTREAM,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
13,19523,DUNCANVILLE,VALUE,2840004768,BAG SNACKS,PRETZELS,FRITO LAY
